# Продвинутая визуализация данных. Домашнее задание 5 (vo_HW)

##### Цель задания. В этом модуле мы будем заниматься практическим использованием методов понижения размерности. В этот раз в качестве зависимой переменной мы будем использовать переменную type — к какому типу относится наше вино.

##### План работы:
EDA.
Поиск корреляций.
Сохранение доли объясненной дисперсии.
Сравнение результатов работы разных алгоритмов понижения размерности.
Сравнение результатов работы алгоритмов анализа данных до и после понижения размерности.
Поиск аномалий в работе алгоритмов.

##### Обзор данных
Содержание. Этот набор данных содержит информацию о трех разных типах вина и ряд их характеристик.

Всего в датасете 13 характеристик: Alcohol, Malic, Ash, Alcalinity, Magnesium, Phenols, Flavanoids, Nonflavanoids, Proanthocyanins, Color, Hue, Dilution, Proline.

Мы будем работать с этим датасетом, используя алгоритмы понижения размерности и машинного обучения.

Давайте импортируем основные библиотеки, которые могут нам понадобиться в рамках работы над датасетом:

In [6]:
pip install catboost

  Obtaining dependency information for catboost from https://files.pythonhosted.org/packages/c8/c0/83de2c76f5b49d0fd71912facf3ed4fe2db06bf225d526883887688b6156/catboost-1.2.3-cp311-cp311-macosx_11_0_universal2.whl.metadata
  Obtaining dependency information for graphviz from https://files.pythonhosted.org/packages/de/5e/fcbb22c68208d39edff467809d06c9d81d7d27426460ebc598e55130c1aa/graphviz-0.20.1-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.2/26.2 MB 3.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.0/47.0 kB 1.2 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install opentsne

  Obtaining dependency information for opentsne from https://files.pythonhosted.org/packages/82/67/dfd7289d035d24d78b4566d0a97b88946c3ae609b0757480ff0f4f5e0d70/openTSNE-1.0.1-cp311-cp311-macosx_10_12_universal2.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 2.8 MB/s eta 0:00:0000:0100:010m

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install umap-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.9/90.9 kB 757.3 kB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for pynndescent>=0.5 from https://files.pythonhosted.org/packages/4e/82/0b9851a2fd4da9b57d7931446f5ebab92a98f1f35d3dc0dae5f9ed50a462/pynndescent-0.5.11-py3-none-any.whl.metadata
  Obtaining dependency information for tqdm from https://files.pythonhosted.org/packages/2a/14/e75e52d521442e2fcc9f1df3c5e456aead034203d4797867980de558ab34/tqdm-4.66.2-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 843.3 kB/s eta 0:00:000:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 1.7 MB/s eta 0:00:00a 0:00:01
  Created wheel for umap-learn: filename=umap_learn-0.5.5-py3-none-any.whl size=86832 sha256=8b73737efe3263140053b5f859f4ff2f3d3b9f1cd1eb7f0d40fbb41fba19c3e7
  Stored in directory: /Users/anastasia/Library/Ca

In [10]:
pip install lightgbm

  Obtaining dependency information for lightgbm from https://files.pythonhosted.org/packages/e3/a9/3bfb8f52fee7ab0950bb94920776dbca00b9515ff5fa504374c059d6192c/lightgbm-4.3.0-py3-none-macosx_10_15_x86_64.macosx_11_6_x86_64.macosx_12_5_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 4.7 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [12]:
# Data manipulation
import pandas as pd
pd.set_option("display.max_columns", None)
import numpy as np

# Data visualization
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import seaborn as sns
import plotly.express as px

# Data preprocessing
from sklearn.model_selection import train_test_split as tts

# Dimensionality Reduction
from openTSNE.sklearn import TSNE
import umap
from sklearn.decomposition import PCA

# Models
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

# Metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_curve, roc_auc_score

# tqdm
from tqdm.auto import tqdm

# warnings
import warnings
warnings.filterwarnings('ignore')

OSError: dlopen(/usr/local/lib/python3.11/site-packages/lightgbm/lib/lib_lightgbm.so, 6): Library not loaded: /usr/local/opt/libomp/lib/libomp.dylib
  Referenced from: /usr/local/lib/python3.11/site-packages/lightgbm/lib/lib_lightgbm.so
  Reason: image not found

In [3]:
df = pd.read_csv('https://raw.githubusercontent.com/ElijahSum/mipt_visualization/master/week_05_visualization/wine.csv')
df.head()

,Type,Alcohol,Malic,Ash,Alcalinity,Magnesium,Phenols,Flavanoids,Nonflavanoids,Proanthocyanins,Color,Hue,Dilution,Proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [4]:
df.shape

(178, 14)

#### Задание 1. Разведочный анализ данных (2 балла)
Воспользуйтесь графиком sns.displot() и постройте 13 графиков распределения разных переменных на одном поле subplots. Параметр hue возьмите равным Type.

Напишите, какие выводы о различиях в типах вин можно сделать исходя из этих графиков.

Критерии оценивания:
Построены графики — 1 балл.
Сделаны выводы о различиях в типах вин — 1 балл.

#### Задание 2. Есть ли корреляции (1 балл)
Постройте sns.heatmap для всех числовых характеристик внутри датасета. Определите, какие три переменные больше всего коррелируют с типом вина. Добавьте стрелочки с текстом (функция annotate) с указанием на эти переменные.

#### Задание 3. Сохранение доли объясненной дисперсии в зависимости от n_components (2 балла)
Воспользуйтесь методом понижения размерности PCA с разным параметром n_components — от 1 до 10. Постройте линейный график совокупной объясненной дисперсии при добавлении каждой новой компоненты.

На том же полотне постройте столбчатую диаграмму: каждый столбец должен показывать, сколько объясненной дисперсии вносит каждая следующая компонента.

Подсказка: аналогичный пример мы разбирали на семинарском занятии.

Критерии оценивания:
Линейный график совокупной объясненной дисперсии — 1 балл.
Столбчатая диаграмма показывает вклад каждой компоненты — 1 балл.

#### Задание 4. Сравнение результатов работы разных алгоритмов понижения размерности (3 балла)
Воспользуйтесь четырьмя любыми алгоритмами понижения размерности (n_components возьмите равным 2 или 3).

Создайте поле с двумерными или трехмерными графиками, на которых будут отображены результаты работы каждого алгоритма, а цветом будет обозначен тип вина (наша зависимая переменная).

Визуально оцените графики и сделайте вывод о том, какой метод отработал лучше.

Опишите результаты и качественно обоснуйте ваш выбор лучшего алгоритма. Если невозможно выбрать лучший алгоритм, также обоснуйте почему.

Критерии оценивания:
Выбранные графики адекватно отражают результаты работы алгоритма — 1 балл.
Описание результатов работы алгоритмов опирается на данные графиков — 1 балл.
Вывод о лучшем алгоритме обоснован — 1 балл.

#### Задание 5. Сравнение результатов работы алгоритмов (4 балла)
Разделите выборку на обучающую и тестовую.

Используйте два метода понижения размерности, которые лучше всего показали себя в предыдущем задании, а также исходные данные без понижения размерности.

Примените 6 любых алгоритмов классификации на этих трех типах данных.

Постройте два sns.heatmap() (для train и test). В каждом из них должно быть по 18 ячеек (6 алгоритмов х 3 вида данных) — в каждой ячейке должно быть указано значение Accuracy для определенного алгоритма с определенными данными. Аналогичную работу мы проделывали на семинаре.

Определите, какой алгоритм дал лучшие результаты и дайте письменное обоснование своего решения:

Как алгоритмы отрабатывают при работе на данных без понижения размерности?
В чем разница в полученных значениях?
Критерии оценивания:
Эксперимент поставлен корректно — 1 балл.
Тепловая карта построена корректно — 1 балл.
Построенные графики визуализируют необходимые зависимости — 1 балл.
Письменное обоснование выбора проведено корректно: не содержит логических ошибок, предположения обоснованны и опираются на данные графиков — 1 балл.

#### Задание 6. Более пристальный анализ (2 балла)
Для лучшего алгоритма из предыдущего задания постройте Classification Report, Confusion Matrix, Precision Recall Curve, ROC Curve.

Удалось ли обнаружить какие-то аномалии в работе нашего алгоритма? Можем ли мы принять полученные результаты?

Критерии оценивания:
По одному баллу за обоснованный ответ на каждый вопрос.